In [1]:
import pandas as pd 
import catboost as cb 
from sklearn.model_selection import train_test_split

In [2]:
train_values = pd.read_csv("train_values_catboost.csv", index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [3]:
test_values = pd.read_csv('test_values_catboost.csv',index_col='building_id')

In [4]:
categoricas = []
for col in train_values.columns:
    if ((col != 'min_geo_id') & (col != 'max_geo_id')):
        categoricas.append(col)
        categoricas.append(train_values.columns.get_loc(col))
        train_values[col] = train_values[col].astype("category")

In [5]:
model = cb.CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1:average=Micro', iterations=3474)

In [6]:
model.fit(train_values, train_labels, categoricas)

Learning rate set to 0.036689
0:	learn: 0.7180287	total: 1.67s	remaining: 1h 36m 47s
1:	learn: 0.7302466	total: 3.31s	remaining: 1h 35m 51s
2:	learn: 0.7298015	total: 4.86s	remaining: 1h 33m 46s
3:	learn: 0.7315244	total: 6.51s	remaining: 1h 34m 6s
4:	learn: 0.7330095	total: 8.32s	remaining: 1h 36m 13s
5:	learn: 0.7331016	total: 10.1s	remaining: 1h 37m 6s
6:	learn: 0.7333318	total: 11.8s	remaining: 1h 37m 33s
7:	learn: 0.7338422	total: 13.4s	remaining: 1h 36m 38s
8:	learn: 0.7349626	total: 15.2s	remaining: 1h 37m 33s
9:	learn: 0.7351929	total: 16.8s	remaining: 1h 37m 6s
10:	learn: 0.7354730	total: 18.6s	remaining: 1h 37m 27s
11:	learn: 0.7360563	total: 20.4s	remaining: 1h 38m 6s
12:	learn: 0.7368237	total: 22.1s	remaining: 1h 38m 6s
13:	learn: 0.7366319	total: 23.8s	remaining: 1h 37m 59s
14:	learn: 0.7370118	total: 25.6s	remaining: 1h 38m 32s
15:	learn: 0.7369350	total: 27.3s	remaining: 1h 38m 18s
16:	learn: 0.7370425	total: 29.1s	remaining: 1h 38m 32s
17:	learn: 0.7372420	total: 30.7s

In [7]:
test_values = pd.read_csv("test_values_catboost.csv", index_col='building_id')

In [8]:
pred = model.predict(test_values) 

In [9]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
submission = pd.DataFrame(data=pred,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [10]:
submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,3
745817,1
421793,3


In [11]:
submission.to_csv('subs/catboost_3474_iterations.csv',index=True)

In [12]:
probas = model.predict_proba(test_values)

In [13]:
probas

array([[2.91610725e-03, 1.75902144e-01, 8.21181749e-01],
       [6.28942833e-04, 9.94729341e-01, 4.64171592e-03],
       [8.47011763e-03, 4.38891676e-02, 9.47640715e-01],
       ...,
       [5.03610453e-02, 9.07357804e-01, 4.22811510e-02],
       [2.92622323e-02, 7.21614004e-01, 2.49123763e-01],
       [6.11143605e-01, 3.21498655e-01, 6.73577398e-02]])

In [14]:
probas_df = pd.DataFrame(data=probas,
                             columns=[1,2,3],
                             index=test_values.index)

In [15]:
probas_df.to_csv('predict_probas/catboost_3474_iterations.csv', index=True)